In [ ]:
# Automated id_dict Boarding Kiosk

In [ ]:
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType

import pandas as pd
from datetime import datetime
import pickle
import io, glob, os, sys, time, uuid

from dotenv import load_dotenv

## Environment

In [ ]:
load_dotenv(override=True)

subscription_key = os.getenv("SUBSCRIPTION_KEY")
location = os.getenv("LOCATION")
account_id = os.getenv("ACCOUNT_ID")

face_endpoint = os.getenv("FACE_ENDPOINT")
face_key = os.getenv("FACE_KEY")

form_endpoint = os.getenv("AZURE_FORM_RECOGNIZER_ENDPOINT")
form_key = os.getenv("AZURE_FORM_RECOGNIZER_KEY")
form_storage = os.getenv("AZURE_STORAGE_BLOB_URL")

In [ ]:
passengers = ['avkash', 'james-jackson', 'james-webb', 'libby-herold', 'radha-s-kumar', 'sameer-kumar']
manifest = pd.read_csv("../material_preparation_step/FlightManifest.csv")


## Initialization

In [ ]:
form_training_client = FormTrainingClient(form_endpoint, AzureKeyCredential(form_key))
form_recognizer_client = FormRecognizerClient(form_endpoint, AzureKeyCredential(form_key))

face_client = FaceClient(face_endpoint, face_key)

training_process = form_training_client.begin_training(trainingDataUrl, use_training_labels=True)
custom_model = training_process.result()

## Person Group

In [ ]:
def build_person_group(client, person_group_id, face_images):
    print("Create and build a person group...")
    # Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
    print("Person group ID:", person_group_id)
    client.person_group.create(person_group_id=person_group_id, name=person_group_id)

    person = client.person_group_person.create(
        person_group_id=person_group_id, name=person_group_id
    )

    for image_p in face_images:
        with open(image_p, "rb") as w:
            client.person_group_person.add_face_from_stream(
                person_group_id, person.person_id, w
            )

    client.person_group.train(person_group_id)

    while True:
        training_status = client.person_group.get_training_status(person_group_id)
        print("Training status: {}.".format(training_status.status))
        if training_status.status is TrainingStatusType.succeeded:
            break
        elif training_status.status is TrainingStatusType.failed:
            client.person_group.delete(person_group_id=person_group_id)
            sys.exit("Training the person group has failed.")
        time.sleep(5)

person_group_id = str(uuid.uuid4())
face_client = FaceClient(face_endpoint, CognitiveServicesCredentials(face_key))
face_images = [file for file in glob.glob("../step3/thumbnails/*.jpg")]

build_person_group(face_client, person_group_id, face_images)

## Validate

In [ ]:
def extract_id_data(id_path):
    with open(id_path, "rb") as f:
        poller = form_recognizer_client.begin_recognize_identity_documents(identity_document=f)

    id_documents = poller.result()[0].to_dict()

    id_dict = {}

    id_dict['first_name'] = id_documents['fields']['FirstName']['value'].strip().lower()
    id_dict['last_name'] = id_documents['fields']['LastName']['value'].strip().lower()
    id_dict['dob'] = id_documents['fields']['DateOfBirth']['value']

    return id_dict


In [ ]:
def extract_boarding_pass(model_id, file_name):
    with open(file_name, "rb") as f:
        poller = form_recognizer_client.begin_recognize_custom_forms(
            model_id=model_id,
            form=f,
            include_field_elements=True,
        )
    boarding_pass = poller.result()[0].to_dict()
    
    boarding_data_dict = {}
    boarding_data_dict['name'] = boarding_pass['fields']['name']['value']
    boarding_data_dict['flight_num'] = boarding_pass['fields']['flight_number']['value']
    boarding_data_dict['seat'] = boarding_pass['fields']['seat']['value']
    boarding_data_dict['from'] = boarding_pass['fields']['from']['value']
    boarding_data_dict['to'] = boarding_pass['fields']['to']['value']
    boarding_data_dict['date'] = datetime.strptime(boarding_pass['fields']['date']['value'][:-3].strip(), "%B %d, %Y")
    boarding_data_dict['time'] = datetime.strptime(boarding_pass['fields']['boarding_time']['value'][:-3].strip(), "%I:%M %p")

    return boarding_data_dict

In [ ]:
id_data = []

for file in os.listdir("../material_preparation_step/digital-ids"):
    if file.endswith(".png"):
        current_data = extract_id_data("../material_preparation_step/digital-ids/" + file)
        id_data.append(current_data)

#with open('id_data', 'wb') as fp:
#     pickle.dump(id_data, fp)


In [ ]:
model_id = "6f95bb00-e6ce-41f2-af7c-fe579deb0c3e"

boarding_data = []

for file in os.listdir("../material_preparation_step/boarding-pass"):
    if file.endswith(".pdf"):
        current_data = extract_boarding_pass(model_id, "../material_preparation_step/boarding-pass/" + file)
        boarding_data.append(current_data)

# with open('boarding_data', 'wb') as fp:
#     pickle.dump(boarding_data, fp)

In [ ]:
def validate_passenger(manifest_df, id_path, boarding_path, luggage):
    # Set initial validation results
    name_validation = False
    dob_validation = False
    boarding_pass_validation = False
    person_id_validation = False
    luggage_validation = False

    # Get data
    id_data = extract_id_data(id_path)
    boarding_data = extract_boarding_pass(model_id, boarding_path)
    manifest_index = manifest_df.index[(manifest_df['First Name'].str.lower() == id_data['first_name']) & (manifest_df['Last Name'].str.lower() == id_data['last_name'])]
    print(manifest_index)

    # 3-Way Person Name Validation
    if (id_data['first_name'] in boarding_data['name'] and id_data['last_name'] in boarding_data['name']):
        name_validation = True
    
    # DoB Validation
    if len(manifest_index > 0):
        'success'
        date_of_birth_id = id_data['dob']
        date_of_birth_manifest = datetime.strptime(manifest_df.iloc[manifest_index]['Date of Birth'][manifest_index[0]], "%d/%m/%Y")
        if (date_of_birth_id.year == date_of_birth_manifest.year and \
            date_of_birth_id.month == date_of_birth_manifest.month and \
                date_of_birth_id.day == date_of_birth_manifest.day):

                manifest_df.loc[manifest_index, 'DoBValidation'] = True
                dob_validation = True

    # Boarding Pass Validation: flight number, seat number, class, origin, destination, flight date, and flight time
    if len(manifest_index > 0):
        manifest_flight_num = manifest_df.iloc[manifest_index]['Flight No.'][manifest_index[0]]
        manifest_seat_num = manifest_df.iloc[manifest_index]['Seat'][manifest_index[0]].strip().lower()
        manifest_origin = manifest_df.iloc[manifest_index]['From'][manifest_index[0]].strip().lower()
        manifest_destination = manifest_df.iloc[manifest_index]['To'][manifest_index[0]].strip().lower()
        manifest_flight_date = datetime.strptime(manifest_df.iloc[manifest_index]['Date'][manifest_index[0]], "%d-%b-%y")
        manifest_flight_time = datetime.strptime(manifest_df.iloc[manifest_index]['Boarding Time'][manifest_index[0]][:-3].strip(), "%I:%M %p")

    if boarding_data['flight_num'] == manifest_flight_num and \
        boarding_data['seat'] == manifest_seat_num and \
            boarding_data['from'] == manifest_origin and \
                boarding_data['to'] == manifest_destination and \
                    boarding_data['date'] == manifest_flight_date and \
                        boarding_data['time'] == manifest_flight_time:

        boarding_pass_validation = True

    with open(id, "rb") as id_file:
        face_client.detect_with_stream(id_file)
        face_ids = [face.face_id for face in face_images]

    identity_result = face_client.identity(face_ids, person_group_id)
    for person in identity_result:
        if len(person.candidates) > 0 and person.candidates[0].confidence >= 0.65:
            person_id_validation = False
            

    manifest.loc[manifest_index, 'NameValidation'] = name_validation
    manifest.loc[manifest_index, 'DoBValidation'] = dob_validation
    manifest.loc[manifest_index, 'PersonValidation'] = person_id_validation
    manifest.loc[manifest_index, 'BoardingPassValidation'] = boarding_pass_validation
    manifest.loc[manifest_index, 'LuggageValidation'] = luggage_validation


    # Save manifest
    manifest.to_csv('manifest.csv')

    # all validation passed
    if (name_validation and dob_validation and boarding_pass_validation and person_id_validation and luggage_validation):
        print(f"""
        Dear {boarding_data['name']},
        You are welcome to flight # {boarding_data['flight_num']} leaving at {boarding_data['time']} from
        {boarding_data['flight_num']} to {boarding_data['to']}.
        Your seat number is {boarding_data['seat']}, and it is confirmed.
        We did not find a prohibited item (lighter) in your carry-on baggage, thanks for following the procedure.
        Your identity is verified so please board the plane.
        """)
    
    elif (name_validation and dob_validation and boarding_pass_validation and person_id_validation and not luggage_validation):
        print(f"""
        Dear {boarding_data['name']},
        You are welcome to flight # {boarding_data['flight_num']} leaving at {boarding_data['time']} from
        {boarding_data['flight_num']} to {boarding_data['to']}.
        Your seat number is {boarding_data['seat']}, and it is confirmed.
        We have found a prohibited item in your carry-on baggage, and it is flagged for removal. 

        Your identity is verified. However, your baggage verification failed, so please see a customer service representative.
        """)
    
    else:
        print("""
        Dear Sir/Madam,
        Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.
        Please see a customer service representative.
        """)
    
    return manifest

In [ ]:
for p in passengers:
    validate_passenger(manifest, "../material_preparation_step/digital-ids/ca-dl-" + p + ".png", "../material_preparation_step/boarding-pass/boarding-" + p + ".pdf", True)